# 03 - Model Explainability

**Objective**: Understand WHY the model makes its predictions.

In [ ]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib

project_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
sys.path.insert(0, str(project_root))
print('Libraries loaded')

In [ ]:
from src.preprocessing import prepare_data
from src.train import create_splits

data_path = project_root / 'data' / 'raw' / 'telco_customer_churn.csv'
X, y, schema = prepare_data(str(data_path))
X_train, X_val, X_test, y_train, y_val, y_test = create_splits(X, y)

pipeline = joblib.load(project_root / 'artifacts' / 'model.joblib')
print(f'Model: {type(pipeline.named_steps["classifier"]).__name__}')

## Feature Importance

In [ ]:
preprocessor = pipeline.named_steps['preprocessor']
classifier = pipeline.named_steps['classifier']

num_features = list(preprocessor.transformers_[0][2])
cat_transformer = preprocessor.transformers_[1][1]
cat_features = list(cat_transformer.named_steps['onehot'].get_feature_names_out(
    preprocessor.transformers_[1][2]
))

feature_names = num_features + cat_features
coefficients = classifier.coef_[0]

importance_df = pd.DataFrame({
    'feature': feature_names,
    'coefficient': coefficients
}).sort_values('coefficient', key=abs, ascending=False)

print('Top 10 Features:')
importance_df.head(10)

In [ ]:
top_n = 15
top_features = importance_df.head(top_n)

fig, ax = plt.subplots(figsize=(10, 8))
colors = ['#e74c3c' if c > 0 else '#2ecc71' for c in top_features['coefficient']]
ax.barh(range(top_n), top_features['coefficient'], color=colors)
ax.set_yticks(range(top_n))
ax.set_yticklabels(top_features['feature'])
ax.invert_yaxis()
ax.axvline(x=0, color='black', linewidth=0.5)
ax.set_title('Feature Importance (Red=Churn, Green=Retain)')
plt.tight_layout()
plt.show()

## Key Insights

**Churn Drivers:**
- Month-to-month contracts
- Fiber optic internet
- Electronic check payment

**Retention Factors:**
- Two-year contracts
- Longer tenure
- Automatic payment methods